In [1]:
using JuMP
using Cbc
using GLPK


## Sets

In [2]:
I=2 # gyn og sarkom
U=2 # 1=hast (og 2=ikke-hast)
S=1 # 1 scanner
D=H=5 # 5 dager
R=2 # 2 radiologer
M=4 # 1=tolking, 2=åpen, 3=scanner, 4=MDT

;

## Parameters

In [3]:
#i rader, m kolonner.
T_im=[[2 1 1 1];
      [2 1 1 1]]

G_d=[2 2 2 2 2]

N_sd=[8 8 8 8 8]

S_sd=[0 0 0 0 1]

D_iuW=[[1 2];
       [1 2]];

K_dr=[[8 8];
      [8 8];
      [8 8];
      [8 8];
      [8 8];]

D_iuD=[[1 1];
       [1 1]]

B_iu=[[2 2];
      [2 2]]

L_d=[1 1 1 1 1]

A_id=[[1 0 0 0 0];
      [0 0 1 0 0]]
;

## Variables

In [4]:
m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, delta_p[1:D] >= 0, Int)
@variable(m, delta_m[1:D] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)

;

## Objective function

In [5]:
@objective(m, Min, sum(delta_p+delta_m));

## Constraints

In [6]:
#Constraint2
@constraint(m,[d=1:D], delta_m[d]-delta_p[d] + sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, r=1:R, m=2)==G_d[d])             
                 
#Constraint3
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)==N_sd[s,d]-S_sd[s,d])
                            
#Constraint4
@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)>=D_iuW[i,u])

#Constraint5
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)>=K_dr[d,r])

#Constraint6
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i,u])

#Constraint7
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#Constraint10
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)-sum(u_i[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D)==0)

#Constraint12
@constraint(m,[d=1:D], sum(w[i,u,d,r,m] for i=1:I, u=1:U, r=1:R, m=3)==sum(N_sd[s,d] for s=1:S))

#Constraint13
@constraint(m,[d=1:D], sum(beta[d,r] for r=1:R)==L_d[d])
                
#Constraint14
@constraint(m,[d=1:D, r=1:R], K_dr[d,r]*(1-beta[d,r])-sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I, u=1:U, m=1:2)==0)

#Constraint15
@constraint(m, [i=1:I,d=1:D], sum(w[i,u,d,r,m] for u=1:U, r=1:R, m=4)==T_im[i,4]*A_id[i,d])
                
#Constraint16
@constraint(m,[i=1:I,d=1:D],sum(gamma[i,d,r] for r=1:R)==A_id[i,d])
                
#Constraint17
@constraint(m,[i=1:I,d=1:D, r=1:R], sum(w[i,u,d,r,m] for u=1:U, m=4)==gamma[i,d,r])
;



## Results

In [7]:
optimize!(m)

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
#println("Dual con1: ", shadow_price(con1[1]))
println("Objective value: ", objective_value(m))
for i=1:I
    for u=1:U
        for s=1:S
            for d=1:D
                println("x[",i,u,s,d,"] = ", value(x[i,u,s,d]))
            end
        end
    end
end
for i=1:I
    for u=1:U
        for d=1:D
            for r=1:R
                for m=1:M
                    println("w[",i,u,d,r,m,"] = ", value(w[i,u,d,r,m]))
                end
            end
        end
    end
end
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    println("u[",i,u,d,h,r,"] = ", value(u_i[i,u,d,h,r]))
                    println("v[",i,u,d,h,r,"] = ", value(v[i,u,d,h,r]))
                end
            end
        end
    end
end

for d =1:D
    println("delta_p[",d,"] = ",value(delta_p[d]))
    println("delta_m[",d,"] = ",value(delta_m[d]))
end

#@variable(m, alpha[1:I,1:U,1:D], Bin)
#@variable(m, beta[1:D,1:R], Bin)
#variable(m, gamma[1:I,1:D,1:R], Bin)

for i=1:I
    for u=1:U
        for d=1:D
            println("alpha[",i,u,d,"] = ", value(alpha[i,u,d]))
        end
    end
end

for d=1:D
    for r=1:R
        println("beta[",d,r,"] = ", value(beta[d,r]))
    end
end
for i=1:I
    for d=1:D
        for r=1:R
            println("gamma[",i,d,r,"] = ", value(gamma[i,d,r]))
        end
    end
end

Status: OPTIMAL
Primal status: FEASIBLE_POINT
Dual status: NO_SOLUTION
Objective value: 0.0
x[1111] = 3.0
x[1112] = 8.0
x[1113] = 8.0
x[1114] = 8.0
x[1115] = 7.0
x[1211] = 2.0
x[1212] = 0.0
x[1213] = 0.0
x[1214] = 0.0
x[1215] = 0.0
x[2111] = 1.0
x[2112] = 0.0
x[2113] = 0.0
x[2114] = 0.0
x[2115] = 0.0
x[2211] = 2.0
x[2212] = 0.0
x[2213] = 0.0
x[2214] = 0.0
x[2215] = 0.0
w[11111] = 0.0
w[11112] = 0.0
w[11113] = 8.0
w[11114] = 1.0
w[11121] = 3.0
w[11122] = 2.0
w[11123] = 0.0
w[11124] = 0.0
w[11211] = 0.0
w[11212] = 0.0
w[11213] = 8.0
w[11214] = 0.0
w[11221] = 3.0
w[11222] = 2.0
w[11223] = 0.0
w[11224] = 0.0
w[11311] = 0.0
w[11312] = 0.0
w[11313] = 8.0
w[11314] = 0.0
w[11321] = 3.0
w[11322] = 2.0
w[11323] = 0.0
w[11324] = 0.0
w[11411] = 0.0
w[11412] = 0.0
w[11413] = 8.0
w[11414] = 0.0
w[11421] = 3.0
w[11422] = 2.0
w[11423] = 0.0
w[11424] = 0.0
w[11511] = 0.0
w[11512] = 0.0
w[11513] = 8.0
w[11514] = 0.0
w[11521] = 3.0
w[11522] = 2.0
w[11523] = 0.0
w[11524] = 0.0
w[12111] = 0.0
w[12112] = 0.

u[22532] = 0.0
v[22532] = 0.0
u[22541] = 0.0
v[22541] = 0.0
u[22542] = 0.0
v[22542] = 0.0
u[22551] = 0.0
v[22551] = 0.0
u[22552] = 0.0
v[22552] = 0.0
delta_p[1] = 0.0
delta_m[1] = 0.0
delta_p[2] = 0.0
delta_m[2] = 0.0
delta_p[3] = 0.0
delta_m[3] = 0.0
delta_p[4] = 0.0
delta_m[4] = 0.0
delta_p[5] = 0.0
delta_m[5] = 0.0
alpha[111] = 1.0
alpha[112] = 0.0
alpha[113] = 0.0
alpha[114] = 0.0
alpha[115] = 0.0
alpha[121] = 1.0
alpha[122] = 0.0
alpha[123] = 0.0
alpha[124] = 0.0
alpha[125] = 0.0
alpha[211] = 1.0
alpha[212] = 0.0
alpha[213] = 0.0
alpha[214] = 0.0
alpha[215] = 0.0
alpha[221] = 1.0
alpha[222] = 0.0
alpha[223] = 0.0
alpha[224] = 0.0
alpha[225] = 0.0
beta[11] = 1.0
beta[12] = 0.0
beta[21] = 1.0
beta[22] = 0.0
beta[31] = 1.0
beta[32] = 0.0
beta[41] = 1.0
beta[42] = 0.0
beta[51] = 1.0
beta[52] = 0.0
gamma[111] = 1.0
gamma[112] = 0.0
gamma[121] = 0.0
gamma[122] = 0.0
gamma[131] = 0.0
gamma[132] = 0.0
gamma[141] = 0.0
gamma[142] = 0.0
gamma[151] = 0.0
gamma[152] = 0.0
gamma[211] = 0.0
gam

## The model

In [8]:
#print(m)